In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report, f1_score

In [3]:
data = pd.read_csv("3_Raisin_Dataset.csv")
y = data.pop("Class")

# Split the data into train and validation, stratifying on the target feature.
X_train, X_val, y_train, y_val = train_test_split(data, y, stratify=y, random_state=42)

In [4]:
# Get a high level overview of the data. This will be useful for slicing.
X_train.describe()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter
count,675.000000,675.000000,675.000000,675.000000,675.000000,675.000000,675.000000
mean,87210.494815,427.650555,254.414345,0.779895,90407.262222,0.701092,1159.625772
std,38388.571707,110.506268,49.752074,0.088938,39602.352484,0.050807,261.820857
min,25387.000000,225.629541,144.618672,0.348730,26139.000000,0.454189,619.074000
25%,59032.500000,343.732369,218.692197,0.740516,61466.500000,0.671134,964.835500
50%,79057.000000,405.936594,247.352044,0.797864,81779.000000,0.709949,1117.107000
75%,103790.500000,493.185891,280.180509,0.840452,108022.500000,0.735886,1302.416500
max,235047.000000,843.956653,492.275279,0.923770,239093.000000,0.830632,2253.557000


In [6]:
lr = LogisticRegression(max_iter=1000, random_state=42)
lb = LabelBinarizer()

# Binarize the target feature.
y_train = lb.fit_transform(y_train)
y_val = lb.transform(y_val)

# Train Logistic Regression.
lr.fit(X_train, y_train.ravel())

LogisticRegression(max_iter=1000, random_state=42)

In [22]:
print(classification_report(y_val, lr.predict(X_val)))

              precision    recall  f1-score   support

           0       0.90      0.86      0.88       112
           1       0.86      0.90      0.88       113

    accuracy                           0.88       225
   macro avg       0.88      0.88      0.88       225
weighted avg       0.88      0.88      0.88       225



In [7]:
f1_score(y_val, lr.predict(X_val), average="weighted")

0.8799288382850028

In [30]:
def sf_f1(X_val, y_val):
    """ Function for calculating the F1 score on slices of the Raisin dataset."""
    for feature in X_val.columns:
        row_slice = X_val[feature] >= X_val[feature].mean()
        f1 = f1_score(y_val[row_slice], lr.predict(X_val[row_slice]), average="weighted")
        print(f"Feature: {feature}")
        print(f"Above meant F1 score: {f1:.4f}")
        # print()
        row_slice = X_val[feature] < X_val[feature].mean()
        f1 = f1_score(y_val[row_slice], lr.predict(X_val[row_slice]), average="weighted")
        print(f"Below meant F1 score: {f1:.4f}")
        print()

In [31]:
sf_f1(X_val, y_val)

Feature: Area
Above meant F1 score: 0.9131
Below meant F1 score: 0.8532

Feature: MajorAxisLength
Above meant F1 score: 0.9001
Below meant F1 score: 0.8313

Feature: MinorAxisLength
Above meant F1 score: 0.8558
Below meant F1 score: 0.9011

Feature: Eccentricity
Above meant F1 score: 0.9118
Below meant F1 score: 0.8292

Feature: ConvexArea
Above meant F1 score: 0.9142
Below meant F1 score: 0.8519

Feature: Extent
Above meant F1 score: 0.8651
Below meant F1 score: 0.8977

Feature: Perimeter
Above meant F1 score: 0.9181
Below meant F1 score: 0.8446



## Model Card 

**Model Details**

Derrick Lewis created the model using using Sci-kit Learn's logistic regression model. Using default hyperparameters from version 1.1.3 except changing model iterations to 1000

**Intended Use**

This model is used to predict the variety of Kecimen and Besni raisin from an image.  

**Metrics**

The model was evaluated using F1 score. The value is 0.87.

**Data**

The data was obtained from the UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/datasets/Raisin+Dataset).

The original data set has 900 rows, and a 75-25 split was used to break this into a train and test set. No stratification was done. To use the data a label binarizer was used on the labels.

**Bias**

Testing for bias was conducted on above and below average metrics, while there was some change in the accuracy of the prediction when slicing the data above and below for each feature, no prominent difference was noticed in the prediction accuracy for any single feature. 